In [14]:
import pandas as pd
import glob
import os
import numpy as np

path = r'../data/curated/merged_dataset/' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
merged_df.drop(['address', 'latitude', 'longitude', 'postcode', 'sa2_2016', 'gdp(USD Millioins)'], axis=1, inplace=True)
merged_df['income_per_person'] = np.log(merged_df['income_per_person'])
merged_df['crime_cases'] = np.log(merged_df['crime_cases'])
merged_df['weekly_rent'] = np.log(merged_df['weekly_rent'])
df = merged_df[merged_df.duplicated(subset=["sa2_2021", "nbed", "nbath", "ncar", "residence_type"], keep=False)]
df = df.dropna()



In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

TARGET_COLS = ['weekly_rent']
COLS = list(df.columns)
df = df[COLS]
df = pd.get_dummies(df, columns=['residence_type', 'sa2_2021', 'nbed', 'nbath', 'ncar'])

train, test = train_test_split(df, train_size=0.8, random_state=0)

X_train, y_train = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]
X_test, y_test = test.drop(TARGET_COLS, axis=1), test[TARGET_COLS]

print(f'{len(X_train)} training instances, {len(X_test)} test instances')

135561 training instances, 33891 test instances


In [16]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Normalization
# from keras.wrappers.scikit_learn import KerasRegressor # Deprecated
from sklearn.ensemble import AdaBoostRegressor
import scikeras
from scikeras.wrappers import KerasRegressor

In [17]:
def simple_model():
    """Base neural network model for AdaBoosting"""
    norm_layer = Normalization()
    norm_layer.adapt(X_train)
    model = keras.Sequential(
        [   
            norm_layer,                   # our normalisation layer recieves the input
            Dense(600, activation='relu'),  # (in case you want to try an extra hidden layer)
            Dense(100, activation='relu'),  # (in case you want to try an extra hidden layer)
            Dense(1, activation='relu')
        ]
    )
    model.compile(
        optimizer='adam',  # Adam optimises using gradient descent, is generally fast and a good choice in many cases
        loss='MSE' # 'sparse_categorical_crossentropy'#'MSE'  # Mean Squared Error makes sense for this problem, 
                    # though we could use Mean Absolute Error, or many other choices.
                    # Classification outputs would use a different loss (eg. BinaryCrossentropy)
    )
    return model


# history = model.fit(
#     x=X_train,
#     y=y_train,
#     batch_size=16,
#     validation_split=0.25,
#     epochs=10
# )

In [18]:
ann_estimator = KerasRegressor(model=simple_model, epochs=3, batch_size=10, verbose=0)
boosted_ann = AdaBoostRegressor(base_estimator= ann_estimator)
boosted_ann.fit(X_train, y_train.values.ravel())# scale your training data 

AdaBoostRegressor(base_estimator=KerasRegressor(batch_size=10, epochs=3, model=<function simple_model at 0x2af403f40>, verbose=0))

In [19]:
# predictions = boosted_ann.predict(X_test.iloc[:])
# errors = np.array(predictions - y_test.iloc[:])
# squared_errors = errors**2
# mean_squared_error = squared_errors.mean()
# 
# print(f'MSE: {mean_squared_error}')
# tot_sum_squares = (np.array(y_test - y_test.mean())**2).sum()
# r2 = 1 - (squared_errors.sum() / tot_sum_squares)
# print(f'Model R^2: {r2:.4f}')

In [21]:
from sklearn.metrics import r2_score
predictions = boosted_ann.predict(X_test.iloc[:])
r2_score(y_test, predictions)

0.35691023649221376